## Import

In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split

sns.set_style("darkgrid")

### Importare dataset

In [190]:
df1 = pd.read_csv('./combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

df1['Rating'] = df1['Rating'].astype(float)

print('Dataset 1 shape:',df1.shape)

# Mi genera un nuovo indice da 1
df1 = df1.reset_index()
print('Dataset 1 shape without rating null:',df1.shape)

print('-Dataset examples-')
print(df1[:5])


Dataset 1 shape: (24058263, 2)
Dataset 1 shape without rating null: (24058263, 3)
-Dataset examples-
   index  Cust_Id  Rating
0      0       1:     NaN
1      1  1488844     3.0
2      2   822109     5.0
3      3   885013     4.0
4      4    30878     4.0


### Prendo i primi 1000

In [187]:
df1 = df1.head(10000)

### Non so cosa faccia ma mi mete movie id 

In [189]:
df_nan = pd.DataFrame(pd.isnull(df1.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df1) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

#print('Movie numpy:',movie_np)
#print('Length:',len(movie_np))

df = df1[pd.notnull(df1['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df)


-Dataset examples-
      index  Cust_Id  Rating  Movie_Id
1         1  1488844     3.0         1
2         2   822109     5.0         1
3         3   885013     4.0         1
4         4    30878     4.0         1
5         5   823519     3.0         1
...     ...      ...     ...       ...
9995   9995  2142408     1.0         8
9996   9996  2231367     3.0         8
9997   9997  1304395     4.0         8
9998   9998  1468830     3.0         8
9999   9999  1369078     1.0         8

[9992 rows x 4 columns]


/tmp/ipykernel_20530/110950279.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Movie_Id'] = movie_np.astype(int)
/tmp/ipykernel_20530/110950279.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cust_Id'] = df['Cust_Id'].astype(int)


### Creo matrice di rating

In [191]:
ratings_matrix = df.pivot(index='Cust_Id', columns='Movie_Id', values='Rating')

# Se vuoi gestire eventuali valori mancanti con uno zero, puoi utilizzare il metodo fillna
ratings_matrix = ratings_matrix.fillna(0)

# Se preferisci avere la matrice come array numpy, puoi utilizzare values
#ratings_matrix_array = ratings_matrix.values

print(ratings_matrix)

Movie_Id    1    2    3    4    5    6    7    8
Cust_Id                                         
685       0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0
695       0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
915       5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
967       0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
1333      0.0  0.0  4.0  0.0  0.0  0.0  0.0  3.0
...       ...  ...  ...  ...  ...  ...  ...  ...
2647871   4.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0
2648122   0.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0
2648204   0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0
2648650   0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0
2648861   0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0

[9619 rows x 8 columns]


## Usare user based NN

#### Calcolo similarità

In [192]:
id_user_target = 30878

In [193]:
user_target = ratings_matrix.loc[id_user_target]
print(user_target)

Movie_Id
1    4.0
2    0.0
3    0.0
4    0.0
5    1.0
6    0.0
7    0.0
8    0.0
Name: 30878, dtype: float64


In [155]:
# Creazione del DataFrame
data = {
    'film': ['Film_A', 'Film_B', 'Film_A', 'Film_B', 'Film_C','Film_A'],
    'rating': [3, 4, 2, 5, 1, 1],
    'utenti': [1,1,2,2,3,3]
}

df = pd.DataFrame(data)

# Visualizzazione del DataFrame
print("DataFrame originale:")
print(df)

# Utilizzo di pivot per ottenere una matrice
matrice = df.pivot(index='utenti', columns='film', values='rating')
matrice = matrice.fillna(0)

# Visualizzazione della matrice
print("\nMatrice ottenuta con pivot:")
print(matrice)

user_target = matrice.loc[1]
print(user_target)


DataFrame originale:
     film  rating  utenti
0  Film_A       3       1
1  Film_B       4       1
2  Film_A       2       2
3  Film_B       5       2
4  Film_C       1       3
5  Film_A       1       3

Matrice ottenuta con pivot:
film    Film_A  Film_B  Film_C
utenti                        
1          3.0     4.0     0.0
2          2.0     5.0     0.0
3          1.0     0.0     1.0
film
Film_A    3.0
Film_B    4.0
Film_C    0.0
Name: 1, dtype: float64


In [194]:
rating_medi = {}
# calcolo rating medio senza considerare gli 0
#for i in ratings_matrix.iterrows():
for i in ratings_matrix.iterrows():
        #print(i[0])
        count = 0
        countn = 0
        for j in i[1].values:
            if j != 0:
                  count += j
                  countn += 1
        rating_medi[i[0]] = count/countn
#print(rating_medi[1333])
print(rating_medi)

{685: 3.0, 695: 1.0, 915: 5.0, 967: 1.0, 1333: 3.5, 2133: 2.0, 2442: 3.0, 3292: 4.0, 3321: 3.5, 3417: 5.0, 3522: 2.0, 3718: 4.0, 3798: 4.0, 4016: 4.0, 4326: 4.0, 4456: 1.0, 4753: 5.0, 4783: 4.0, 5225: 4.0, 5749: 5.0, 6154: 4.0, 6303: 3.0, 6384: 3.0, 6460: 5.0, 6510: 3.0, 6689: 4.0, 7116: 1.0, 7284: 5.0, 7798: 3.5, 8012: 4.0, 8117: 5.0, 8121: 1.0, 8212: 4.0, 8753: 3.0, 8999: 2.0, 9660: 3.0, 10251: 4.0, 10612: 1.0, 10972: 3.0, 10996: 2.0, 11169: 2.0, 11186: 4.0, 11194: 3.0, 11409: 5.0, 11589: 3.0, 13432: 3.0, 13511: 5.0, 13561: 3.0, 13651: 3.0, 14035: 5.0, 14326: 4.0, 14389: 3.0, 14422: 2.0, 14728: 2.0, 14756: 4.0, 14924: 5.0, 15218: 5.0, 15288: 5.0, 15420: 2.0, 15796: 2.0, 16121: 1.0, 16229: 4.0, 16272: 3.3333333333333335, 16818: 3.0, 17063: 5.0, 17088: 5.0, 17149: 1.0, 17510: 5.0, 17890: 3.0, 18753: 1.0, 18960: 2.0, 19006: 5.0, 19106: 2.0, 19289: 5.0, 20936: 1.0, 21153: 5.0, 21296: 4.0, 21368: 5.0, 21619: 2.0, 21722: 3.5, 21777: 3.0, 21983: 3.0, 22015: 4.0, 22178: 5.0, 22575: 5.0, 2266

In [195]:
# Definire la funzione per il calcolo della similarità tra utenti utilizzando la person coefficent
similarità = {}
def user_similarity(ratings,user_target):
    # scorro UTENTI
    for i in ratings.iterrows():
        # se l'UTENTE è diverso dall'utente target
        if i[0] != id_user_target:
            # calcolo similarità
            #print(i[1].values[0])
            numeratore = 0
            denominatoreP1 = 0
            denominatoreP2 = 0
            for j in i[1].keys():
                numeratore += (i[1][j]-rating_medi[i[0]])*(user_target[j]-rating_medi[id_user_target])
                
                denominatoreP1 += (i[1][j]-rating_medi[i[0]])**2
                
                denominatoreP2 +=(user_target[j]-rating_medi[id_user_target])**2
                
            sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))
            similarità[(i[0],id_user_target)] = sim

# Calcolare la similarità tra utenti
user_similarity(ratings_matrix,user_target)
#user_similarity(matrice,user_target)
print(similarità)

/tmp/ipykernel_20530/117669305.py:20: RuntimeWarning: invalid value encountered in double_scalars
  sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))


{(685, 30878): 0.7873359887517357, (695, 30878): 0.7290148043997553, (915, 30878): 0.9622995418076771, (967, 30878): 0.7290148043997553, (1333, 30878): 0.6278090029174034, (2133, 30878): 0.7290148043997553, (2442, 30878): 0.962299541807677, (3292, 30878): 0.7290148043997553, (3321, 30878): 0.9148074042510734, (3417, 30878): 0.7290148043997554, (3522, 30878): 0.7290148043997553, (3718, 30878): 0.7290148043997553, (3798, 30878): 0.7290148043997553, (4016, 30878): 0.7290148043997553, (4326, 30878): 0.9622995418076771, (4456, 30878): 0.7290148043997553, (4753, 30878): 0.7290148043997554, (4783, 30878): 0.7290148043997553, (5225, 30878): 0.7290148043997553, (5749, 30878): 0.7290148043997554, (6154, 30878): 0.7290148043997553, (6303, 30878): 0.7290148043997553, (6384, 30878): 0.7290148043997553, (6460, 30878): 0.7290148043997554, (6510, 30878): 0.7290148043997553, (6689, 30878): 0.7290148043997553, (7116, 30878): 0.7290148043997553, (7284, 30878): 0.7290148043997554, (7798, 30878): 0.6278090

In [197]:
utenteSimileUno = ratings_matrix.loc[915]
utenteSimileDue = ratings_matrix.loc[685]
print(utenteSimileUno)
print(utenteSimileDue)
print(user_target)


Movie_Id
1    5.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
Name: 915, dtype: float64
Movie_Id
1    0.0
2    0.0
3    0.0
4    0.0
5    3.0
6    0.0
7    0.0
8    0.0
Name: 685, dtype: float64
Movie_Id
1    4.0
2    0.0
3    0.0
4    0.0
5    1.0
6    0.0
7    0.0
8    0.0
Name: 30878, dtype: float64


In [201]:
# FACCIO UNA SOGLIO K DI VICINATO SULLA BASE DELLA SIMILARITÀ
sim_soglia = {}
for i in similarità:
    if similarità[i] > 0.97:
        sim_soglia[i] = similarità[i]
print(sim_soglia)

{(494609, 30878): 0.9761904761904762, (504620, 30878): 0.989743318610787, (767518, 30878): 0.9798272520870257}


#### Prediction

In [183]:
# ITEM PER IL QUALE VOGLIO PREDIRRE RATING
id_item_target = 1

In [202]:
# Definire una funzione per la predizione delle valutazioni utilizzando User-Based Collaborative Filtering
def predict_user_based(item,user_similarities,ratings_matrix,rating_medi,id_user_target):
    # Calcolare le valutazioni previste
    num = 0
    den = 0
    pred_ratings = 0
    # verifico che per quell'item il rating sia 0
    if ratings_matrix.loc[id_user_target].values[item] == 0:
        for i in user_similarities:
            #if ratings_matrix.loc[i[0]].values[item] != 0:
            num += (user_similarities[i]* (ratings_matrix.loc[i[0]].values[item]-rating_medi[i[0]]))
            den += user_similarities[i]
        pred_ratings = rating_medi[id_user_target] + (num / den)
        return pred_ratings
    else:
        return ratings_matrix.loc[id_user_target].values[item]

# Ottenere le previsioni per tutte le valutazioni
#predicted_ratings_user_based = predict_user_based(id_item_target, sim_soglia,ratings_matrix,rating_medi,id_user_target)
#print(f'Rating per l item {id_item_target} è di {predicted_ratings_user_based} per l utente {id_user_target}')


### Ranking

In [204]:
# Ottenere le previsioni per tutte le valutazioni
for m in range(len(user_target)):
    predicted_ratings_user_based = predict_user_based(m, sim_soglia, ratings_matrix,rating_medi,id_user_target)
    if predicted_ratings_user_based > 0:    
        print(f'Rating per l item {m} è di {predicted_ratings_user_based} per l utente {id_user_target}')
for m in range(len(user_target)):
    if user_target.values[m] > 0:
        print(f'Rating ESPLICITI per l item {m} è di {user_target.values[m]} per l utente {id_user_target}')


Rating per l item 0 è di 4.0 per l utente 30878
Rating per l item 1 è di 0.1696776094908521 per l utente 30878
Rating per l item 2 è di 0.1696776094908521 per l utente 30878
Rating per l item 3 è di 0.1696776094908521 per l utente 30878
Rating per l item 4 è di 1.0 per l utente 30878
Rating per l item 5 è di 0.5023003974532534 per l utente 30878
Rating per l item 6 è di 0.1696776094908521 per l utente 30878
Rating per l item 7 è di 0.5010658180563976 per l utente 30878
Rating ESPLICITI per l item 0 è di 4.0 per l utente 30878
Rating ESPLICITI per l item 4 è di 1.0 per l utente 30878


In [48]:
# Load the MovieLens dataset
reader = Reader()
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:],reader)

# Create a user-based collaborative filtering model
sim_options = {
    'name': 'cosine',
    'user_based': True  # Compute user similarity
}

# Initialize the KNNBasic algorithm
model = KNNBasic(sim_options=sim_options)

# Perform cross-validation
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0398  1.0926  1.0525  1.1017  1.1319  1.0837  0.0335  
MAE (testset)     0.8637  0.8745  0.8583  0.8581  0.9345  0.8778  0.0290  
Fit time          0.05    0.03    0.02    0.01    0.02    0.02    0.01    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.03982522, 1.09262128, 1.0524882 , 1.1017375 , 1.13190181]),
 'test_mae': array([0.86371016, 0.87452112, 0.85830783, 0.85812017, 0.93454113]),
 'fit_time': (0.04631924629211426,
  0.026477336883544922,
  0.015208959579467773,
  0.014096975326538086,
  0.01614999771118164),
 'test_time': (0.003204345703125,
  0.0025000572204589844,
  0.0025870800018310547,
  0.003971099853515625,
  0.004466056823730469)}

In [49]:
# Load the dataset
#data = Dataset.load_builtin('ml-100k')
# Split the data into a training set and a test set
trainset, testset = train_test_split(data, test_size=0.25)
# Initialize and fit the model
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)
# Make predictions
predictions = model.test(testset)
# Calculate RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0605
RMSE: 1.0605386956375422


## Usare item based NN 

In [50]:
# Load the MovieLens dataset
reader = Reader()
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:],reader)

# Create a user-based collaborative filtering model
sim_options = {
    'name': 'cosine',
    'item_based': True  # Compute user similarity
}

# Initialize the KNNBasic algorithm
model = KNNBasic(sim_options=sim_options)

# Perform cross-validation
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1204  1.0306  0.9909  1.0413  1.1874  1.0741  0.0705  
MAE (testset)     0.9110  0.8565  0.8078  0.8497  0.9434  0.8737  0.0479  
Fit time          0.04    0.02    0.02    0.02    0.02    0.02    0.01    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.12035741, 1.03064277, 0.99089216, 1.041291  , 1.18741889]),
 'test_mae': array([0.91098243, 0.85650486, 0.8077669 , 0.84971222, 0.94342011]),
 'fit_time': (0.04016304016113281,
  0.02325892448425293,
  0.02111673355102539,
  0.016490936279296875,
  0.017423152923583984),
 'test_time': (0.003538846969604492,
  0.0025937557220458984,
  0.003205537796020508,
  0.003868579864501953,
  0.004126787185668945)}

In [51]:
# Load the dataset
data = Dataset.load_builtin('ml-100k')
# Split the data into a training set and a test set
trainset, testset = train_test_split(data, test_size=0.25)
# Initialize and fit the model
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)
# Make predictions
predictions = model.test(testset)
# Calculate RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0166
RMSE: 1.0166491829442186


In [205]:
ratings_matrix_transpose = df.pivot(index='Movie_Id', columns='Cust_Id', values='Rating')

# Se vuoi gestire eventuali valori mancanti con uno zero, puoi utilizzare il metodo fillna
ratings_matrix_transpose = ratings_matrix_transpose.fillna(0)

# Se preferisci avere la matrice come array numpy, puoi utilizzare values
#ratings_matrix_array = ratings_matrix.values

print(ratings_matrix_transpose)

Cust_Id   685      695      915      967      1333     2133     2442     \
Movie_Id                                                                  
1             0.0      0.0      5.0      0.0      0.0      0.0      3.0   
2             0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3             0.0      0.0      0.0      0.0      4.0      0.0      0.0   
4             0.0      0.0      0.0      1.0      0.0      0.0      0.0   
5             3.0      0.0      0.0      0.0      0.0      0.0      0.0   
6             0.0      0.0      0.0      0.0      0.0      0.0      0.0   
7             0.0      0.0      0.0      0.0      0.0      0.0      0.0   
8             0.0      1.0      0.0      0.0      3.0      2.0      0.0   

Cust_Id   3292     3321     3417     ...  2647026  2647078  2647197  2647532  \
Movie_Id                             ...                                       
1             0.0      3.0      0.0  ...      0.0      0.0      0.0      0.0   
2        

In [224]:
print(ratings_matrix_transpose[5])

KeyError: 5

In [206]:
rating_medi_transpose = {}
# calcolo rating medio senza considerare gli 0
for i in ratings_matrix_transpose.iterrows():
        #print(i[0])
        count = 0
        countn = 0
        for j in i[1].values:
            if j != 0:
                  count += j
                  countn += 1
        rating_medi_transpose[i[0]] = count/countn
print(rating_medi_transpose)

{1: 3.749542961608775, 2: 3.5586206896551724, 3: 3.6411530815109345, 4: 2.73943661971831, 5: 3.9192982456140353, 6: 3.084396467124632, 7: 2.129032258064516, 8: 3.1873722926031878}


In [207]:
item_target = ratings_matrix_transpose.loc[id_item_target]
print(item_target)

Cust_Id
685        0.0
695        0.0
915        5.0
967        0.0
1333       0.0
          ... 
2647871    4.0
2648122    0.0
2648204    0.0
2648650    0.0
2648861    0.0
Name: 1, Length: 9619, dtype: float64


In [220]:
# Definire la funzione per il calcolo della similarità tra utenti utilizzando la person coefficent
similarita_transpose = {}
def item_similarity(ratings,item_target):
    # scorro UTENTI
    for i in ratings.iterrows():
        # se l'ITEM è diverso dall'item target
        #print(i[1].keys())
        if i[0] != id_item_target:
            # calcolo similarità
            #print(i[1].values[0])
            numeratore = 0
            denominatoreP1 = 0
            denominatoreP2 = 0
            for j in i[1].keys():
                #print(item_target.values[j])
                numeratore += (i[1][j]-rating_medi_transpose[i[0]])*(item_target[j]-rating_medi_transpose[id_item_target])
                denominatoreP1 += (i[1][j]-rating_medi_transpose[i[0]])**2
                denominatoreP2 +=(item_target[j]-rating_medi_transpose[id_item_target])**2
            sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))
            similarita_transpose[(i[0],id_item_target)] = sim

# Calcolare la similarità tra utenti
item_similarity(ratings_matrix_transpose,item_target)
print(similarita_transpose)

{(2, 1): 0.9601287685350038, (3, 1): 0.8410682435295455, (4, 1): 0.959913938553727, (5, 1): 0.8980376398331075, (6, 1): 0.901087208897411, (7, 1): 0.9634148997830742, (8, 1): 0.5889356937533805}


In [239]:
itemSimileUno = ratings_matrix_transpose.loc[4]
itemSimileDue = ratings_matrix_transpose.loc[8]
print(itemSimileUno)
print(itemSimileDue)
print(item_target)

Cust_Id
685        0.0
695        0.0
915        0.0
967        1.0
1333       0.0
          ... 
2647871    0.0
2648122    0.0
2648204    0.0
2648650    0.0
2648861    0.0
Name: 4, Length: 9619, dtype: float64
Cust_Id
685        0.0
695        1.0
915        0.0
967        0.0
1333       3.0
          ... 
2647871    0.0
2648122    0.0
2648204    3.0
2648650    0.0
2648861    0.0
Name: 8, Length: 9619, dtype: float64
Cust_Id
685        0.0
695        0.0
915        5.0
967        0.0
1333       0.0
          ... 
2647871    4.0
2648122    0.0
2648204    0.0
2648650    0.0
2648861    0.0
Name: 1, Length: 9619, dtype: float64


In [226]:
# FACCIO UNA SOGLIO K DI VICINATO SULLA BASE DELLA SIMILARITÀ
sim_soglia_transpose = {}
for i in similarita_transpose:
    if similarita_transpose[i] > 0.96:
        sim_soglia_transpose[i] = similarita_transpose[i]
print(sim_soglia_transpose)

{(2, 1): 0.9601287685350038, (7, 1): 0.9634148997830742}


In [240]:
# Definire una funzione per la predizione delle valutazioni utilizzando User-Based Collaborative Filtering
def predict_item_based(user,item_similarities,ratings_matrix_transpose,id_item_target):
    # Calcolare le valutazioni previste
    num = 0
    den = 0
    if ratings_matrix_transpose.loc[id_item_target][user] == 0:
        for i in item_similarities:
            num += (item_similarities[i]* (ratings_matrix_transpose.loc[i[0]][user]))
            den += item_similarities[i]
        pred_ratings = (num / den)
        return pred_ratings
    else:
        return ratings_matrix_transpose.loc[id_item_target][user]

# Ottenere le previsioni per tutte le valutazioni
#predicted_ratings_user_based = predict_item_based(id_item_target, similarita_transpose,ratings_matrix_transpose,id_item_target)
#print(f'Rating per l item {id_item_target} è di {predicted_ratings_user_based} per l utente {id_user_target}')


In [241]:
# Ottenere le previsioni per tutte le valutazioni
for m in item_target.keys():
    predicted_ratings_user_based = predict_item_based(m, sim_soglia_transpose,ratings_matrix_transpose,id_item_target)
    if predicted_ratings_user_based > 0:    
        print(f'Rating per l item {m} è di {predicted_ratings_user_based} per l utente {id_user_target}')
for m in range(len(user_target)):
    if user_target.values[m] > 0:
        print(f'Rating ESPLICITI per l item {m} è di {user_target.values[m]} per l utente {id_user_target}')

Rating per l item 915 è di 5.0 per l utente 30878
Rating per l item 2442 è di 3.0 per l utente 30878
Rating per l item 3321 è di 3.0 per l utente 30878
Rating per l item 4326 è di 4.0 per l utente 30878
Rating per l item 11409 è di 2.4957290659653393 per l utente 30878
Rating per l item 11589 è di 3.0 per l utente 30878
Rating per l item 13651 è di 3.0 per l utente 30878
Rating per l item 14756 è di 4.0 per l utente 30878
Rating per l item 14924 è di 5.0 per l utente 30878
Rating per l item 16272 è di 4.0 per l utente 30878
Rating per l item 21722 è di 4.0 per l utente 30878
Rating per l item 30245 è di 5.0 per l utente 30878
Rating per l item 30878 è di 4.0 per l utente 30878
Rating per l item 31913 è di 4.0 per l utente 30878
Rating per l item 34907 è di 3.0 per l utente 30878
Rating per l item 38052 è di 3.0 per l utente 30878
Rating per l item 41422 è di 1.9965832527722713 per l utente 30878
Rating per l item 42921 è di 3.0 per l utente 30878
Rating per l item 42930 è di 3.0 per l 

## Paradigma : User based vs item base

## Matrix factorization: SVD 

In [62]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:], reader)
#data.split(n_folds=3)
trainsetData, testsetData = train_test_split(data, test_size=0.2)

In [63]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'],verbose=True,return_train_measures=True,n_jobs=100)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0817  1.1257  1.0821  0.9862  1.1475  1.0846  0.0554  
MAE (testset)     0.8892  0.8968  0.8801  0.8011  0.9095  0.8753  0.0383  
RMSE (trainset)   0.7181  0.7087  0.6974  0.7145  0.7284  0.7134  0.0103  
MAE (trainset)    0.5703  0.5643  0.5540  0.5749  0.5757  0.5679  0.0080  
Fit time          0.02    0.02    0.02    0.02    0.02    0.02    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.08172015, 1.12566475, 1.08206551, 0.98620392, 1.14747006]),
 'train_rmse': array([0.71806266, 0.70865702, 0.69741135, 0.71451957, 0.72844242]),
 'test_mae': array([0.88921889, 0.89675019, 0.88007379, 0.80112606, 0.90950042]),
 'train_mae': array([0.57027905, 0.56429528, 0.55403185, 0.57491372, 0.57573713]),
 'fit_time': (0.024950742721557617,
  0.022351980209350586,
  0.023331403732299805,
  0.0203859806060791,
  0.0207977294921875),
 'test_time': (0.0018596649169921875,
  0.002742290496826172,
  0.0024137496948242188,
  0.0017681121826171875,
  0.0017161369323730469)}

## Metriche : Precision vs Recal vs F1 vs MAE vs RMSE vs NMAE

In [64]:
ts = data.build_full_trainset()
svd.fit(ts)

predictions = svd.test(testsetData)

y_true = [int(pred.r_ui) for pred in predictions]
y_pred = [int(pred.est) for pred in predictions]
print(y_true)
print(y_pred)


# Calcola le metriche aggiuntive
precision = precision_score(y_true, [round(pred) for pred in y_pred], average='micro')
recall = recall_score(y_true, [round(pred) for pred in y_pred], average='micro')
f1 = f1_score(y_true, [round(pred) for pred in y_pred], average='micro')
nmae = accuracy.mae(predictions) / (max(y_true) - min(y_true))
mae = accuracy.mae(predictions)

# Stampa i risultati
print(f'Precision:',precision)
print(f'Recall:',recall)
print(f'F1 Score:',f1)
print(f'NMAE:',nmae)
print(f'MAE:',mae)

[5, 4, 3, 4, 3, 4, 4, 3, 4, 3, 4, 5, 5, 4, 4, 1, 4, 3, 3, 3, 1, 5, 4, 3, 4, 5, 5, 4, 3, 3, 4, 3, 4, 3, 4, 3, 5, 3, 4, 5, 1, 3, 3, 2, 2, 4, 4, 5, 3, 4, 1, 4, 5, 4, 4, 3, 2, 3, 3, 5, 4, 4, 5, 1, 5, 1, 3, 2, 1, 4, 5, 1, 4, 5, 3, 3, 5, 1, 3, 3, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 5, 5, 3, 4, 4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 5, 3, 5, 4, 5, 2, 3, 1, 5, 4, 4, 5, 5, 3, 3, 1, 3, 5, 5, 4, 3, 4, 5, 4, 5, 3, 4, 4, 3, 5, 3, 5, 3, 4, 4, 5, 5, 5, 4, 4, 3, 3, 1, 4, 3, 4, 4, 5, 4, 3, 4, 3, 4, 5, 4, 1, 4, 3, 3, 3, 2, 2, 5, 4, 4, 4, 5, 4, 5, 3, 4, 3, 4, 3, 2, 4, 1, 4, 3, 5, 5, 4, 4, 5, 5, 5, 1, 2, 3, 4, 5, 4, 3, 4, 4]
[4, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 4, 3, 4, 2, 3, 3, 3, 3, 2, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 4, 2, 3, 3, 3, 2, 3, 3, 4, 3, 3, 2, 4, 4, 3, 3, 3, 2, 3, 3, 4, 4, 4, 4, 2, 4, 2, 3, 3, 2, 4, 4, 2, 3, 4, 3, 3, 4, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 4, 3, 4, 3, 3, 2, 4, 3, 3, 4, 4, 3, 3, 2, 3, 4, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3,

Probabilistic approch : Classificatore bayesiano 